In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import numba
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn import linear_model

### Features:

Age, Gender, Occupation, Zipcode, Genre

In [27]:
ratings.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [28]:
# Load data
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python')
users = pd.read_csv('ml-1m/users.dat', sep='::', engine='python')
movies = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python')

ratings.columns = map(str.lower, ratings.columns)
users.columns = map(str.lower, users.columns)
movies.columns = map(str.lower, movies.columns)


df = pd.merge(ratings,users)
df = pd.merge(df,movies)
df['gender'] = df['gender'].replace(['M','F'],[0,1])
df['genres'] = df['genres'].str.replace('|',',')
df['genres'] = df['genres'].str.replace('Action','0')
df['genres'] = df['genres'].str.replace('Adventure','1')
df['genres'] = df['genres'].str.replace('Animation','2')
df['genres'] = df['genres'].str.replace("Children's",'3')
df['genres'] = df['genres'].str.replace('Comedy','4')
df['genres'] = df['genres'].str.replace('Crime','5')
df['genres'] = df['genres'].str.replace('Documentary','6')
df['genres'] = df['genres'].str.replace('Drama','7')
df['genres'] = df['genres'].str.replace('Fantasy','8')
df['genres'] = df['genres'].str.replace('Film-Noir','9')
df['genres'] = df['genres'].str.replace('Horror','10')
df['genres'] = df['genres'].str.replace('Musical','11')
df['genres'] = df['genres'].str.replace('Mystery','12')
df['genres'] = df['genres'].str.replace('Romance','13')
df['genres'] = df['genres'].str.replace('Sci-Fi','14')
df['genres'] = df['genres'].str.replace('Thriller','15')
df['genres'] = df['genres'].str.replace('War','16')
df['genres'] = df['genres'].str.replace('Western','17')


df['age'] = df['age'].replace('1',0)
df['age'] = df['age'].replace('18',1)
df['age'] = df['age'].replace('25',2)
df['age'] = df['age'].replace('35',3)
df['age'] = df['age'].replace('45',4)
df['age'] = df['age'].replace('50',5)
df['age'] = df['age'].replace('56',6)

In [29]:
df.head()

,userid,movieid,rating,timestamp,gender,age,occupation,zip-code,title,genres
0,1,1193,5,978300760,1,1,10,48067,One Flew Over the Cuckoo's Nest (1975),7
1,2,1193,5,978298413,0,56,16,70072,One Flew Over the Cuckoo's Nest (1975),7
2,12,1193,4,978220179,0,25,12,32793,One Flew Over the Cuckoo's Nest (1975),7
3,15,1193,4,978199279,0,25,7,22903,One Flew Over the Cuckoo's Nest (1975),7
4,17,1193,5,978158471,0,50,1,95350,One Flew Over the Cuckoo's Nest (1975),7


In [33]:
n_users = users['userid'].max(); print("Number of users: {0}".format(n_users))
n_items = movies['movieid'].max(); print("Number of items: {0}".format(n_items))

Number of users: 6040
Number of items: 3952


### Building the features matrix

In [37]:
users = df['userid'].values-1
movies = df['movieid'].values-1
rating = df['rating'].values
gender = df['gender'].values
age = df['age'].values
occupation = df['occupation'].values
genres = df['genres'].values

In [45]:
feat_matrix = np.zeros((n_users,n_items,48))

uid = ratings['userid'].values - 1
mid = ratings['movieid'].values - 1
rt =  ratings['rating'].values

for i in range(df.shape[0]):
    # Check gender
    u_index = uid[i]
    m_index = mid[i]
    
    if gender[i] == 0:
        feat_matrix[u_index, m_index, 0]=1
    else:
        feat_matrix[u_index, m_index ,1]=1

    # Check age
    for j in range(0,6):
        if age[i] == j:
            feat_matrix[u_index, m_index, j+2] = 1

    # Check occupation
    for k in range(0,20):
        if occupation[i] == k:
            feat_matrix[u_index, m_index, k+9] = 1

    # Check genres
    for l in range(0,17):
        if '%d' %(l) in genres[i]:
            feat_matrix[u_index, m_index, l+30]=1       

In [31]:
uid = ratings['userid'].values - 1
mid = ratings['movieid'].values - 1
rt =  ratings['rating'].values

In [12]:
@numba.jit()
def _error_function(theta,ratings):
    prod = np.dot(feature_vector,theta)
    dif = [(x - prod)**2. for x in ratings]
    return sum((dif))

def error_function(theta,ratings):
    prod = np.dot(feature_vector,theta)
    dif = [(x - prod)**2. for x in ratings]
    return sum((dif))

NameError: name 'numba' is not defined

In [21]:
feat_matrix = np.zeros((n_users,48))

for i in range(n_users):
    '''Loop over all users'''
    
    user = df.loc[df['userid'] == i] # User data frame
    n = user.shape[0] # Number of reviews
    gender = user['gender'].values[0]
    occupation = user['occupation'].values[0]
    ratings = user['rating'].values
    age = user['age'].values[0]
    genres = user['genres'].values[0]
    
    # Check gender
    if gender == 0:
        feat_matrix[i,0]=1
    else:
        feat_matrix[i,1]=1
    
    # Check age
    for j in range(0,6):
        if age == j:
            feat_matrix[i,j+2] = 1
    
    # Check occupation
    for k in range(0,20):
        if occupation == k:
            feat_matrix[i,k+9] = 1
            
    # Check genres
    for l in range(0,17):
        if '%d' %(l) in genres:
            feat_matrix[i,l+30]=1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
n_users

6040

In [394]:
n_users = df['userid'].unique().shape[0] # Total number of unique users
userid_label = df['userid'].unique()

feat_matrix = np.zeros(())

theta_hats = []
for i in range(1,6040):
    
    '''Loop over all users'''
    
    user = df.loc[df['userid'] == i] # User data frame
    
    #train,test = train_test_split(user,test_size=0.2)
    
    n = user.shape[0] # Number of reviews
    gender = user['gender'].values[0]
    occupation = user['occupation'].values[0]
    ratings = user['rating'].values
    age = user['age'].values[0]
    genres = user['genres'].values[0]
    
    
    feature_vector = np.zeros(48)
    
    # Check gender
    if gender == 0:
        feature_vector[0]=1
    else:
        feature_vector[1]=1
    
    # Check age
    for j in range(0,6):
        if age == j:
            feature_vector[j+2] = 1
    
    # Check occupation
    for k in range(0,20):
        if occupation == k:
            feature_vector[k+9] = 1
            
    # Check genres
    for l in range(0,17):
        if '%d' %(l) in genres:
            feature_vector[l+30]=1
    
    #loss_function = []


    theta_0 = np.ones(48)
    res = minimize(error_function,theta_0,ratings)
    theta_hats.append(res.x)
    

    #reg = linear_model.LinearRegression()
    #reg.fit(ratings.reshape(-1, 1),feature_vector)
    #predict = reg.predict(test)
    

KeyboardInterrupt: 

In [390]:
ratings.shape

(53,)

In [253]:
feature_vector[0]=1.

In [254]:
feature_vector

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])